In [1]:
import pandas as pd
import numpy as np
import wrangle as w

pd.set_option('display.max_rows', 500)

In [2]:
df = w.acquire_anonymized_curriculum_access_data()

/Users/fostermark/codeup-data-science/project-5_anomaly_detection/wrangle.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None,


In [3]:
df

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2.0
...,...,...,...,...,...,...,...,...,...,...
900218,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900219,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900220,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,2.0
900221,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,Staff,2014-02-04,2014-02-04,2.0


In [4]:
# 1 - PHP
# 2 - Java
# 3 - Data Science
# 4 - Front End

In [5]:
df = w.clean_the_data(df)

In [6]:
df 

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
date,,,,,,,,,,
2018-01-26,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,PHP
2018-01-26,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,PHP
2018-01-26,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,PHP
2018-01-26,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,PHP
2018-01-26,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,Java
...,...,...,...,...,...,...,...,...,...,...
2021-04-21,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,Java
2021-04-21,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,Java
2021-04-21,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,Staff,2014-02-04,2014-02-04,Java


## 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

There are 

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 to 2021-04-21
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        900223 non-null  datetime64[ns]
 1   time        900223 non-null  object        
 2   page        900222 non-null  object        
 3   user_id     900223 non-null  int64         
 4   cohort_id   847330 non-null  float64       
 5   ip          900223 non-null  object        
 6   name        847330 non-null  object        
 7   start_date  847330 non-null  datetime64[ns]
 8   end_date    847330 non-null  datetime64[ns]
 9   program_id  900223 non-null  object        
dtypes: datetime64[ns](3), float64(1), int64(1), object(5)
memory usage: 75.5+ MB


In [8]:
#change df name!
active_student_access = df.query('start_date < date < end_date')

In [54]:

total_users = df.user_id.nunique()

981

In [53]:
active_users = active_student_access.user_id.nunique()

724

In [ ]:
#

In [13]:
usage = active_student_access.user_id.value_counts().sort_values()

In [55]:
#I defined "hardly active" as those students who accessed less than or equal to 10% of the average usage (pages viewed)
usage.mean() * .10

89.00359116022099

In [56]:
usage[usage <= 89]

879     1
679     3
832     3
278     4
956     5
812     7
388     8
572    12
889    16
968    16
487    17
963    17
961    18
772    20
49     25
64     25
976    25
24     26
41     28
972    31
973    32
967    33
529    35
959    38
971    41
964    42
371    42
981    42
960    45
741    47
978    49
965    51
56     52
962    52
543    53
46     54
57     62
969    62
858    63
45     65
36     65
975    66
250    67
65     73
52     75
977    76
974    76
13     79
33     79
955    80
702    85
806    88
954    88
253    89
Name: user_id, dtype: int64

In [9]:
#278 is weird because they only started accessing the curriculum when the cohort was almost over
#Those who started accessing the curriculum when the cohort was almost over
#Java students
#278, 812
active_student_access[active_student_access.user_id == 278]

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
date,,,,,,,,,,
2018-09-27,2018-09-27,13:57:44,/,278,24.0,97.105.19.58,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-27,2018-09-27,14:47:37,java-ii/arrays,278,24.0,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-27,2018-09-27,14:58:48,java-ii/arrays,278,24.0,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-27,2018-09-27,14:59:07,java-ii/collections,278,24.0,107.77.217.9,Voyageurs,2018-05-29,2018-10-11,Java


In [41]:
active_student_access[active_student_access.user_id == 250]

,date,time,page,user_id,cohort_id,ip,name,start_date,end_date,program_id
date,,,,,,,,,,
2018-09-07,2018-09-07,08:58:55,html-css/elements,250,24.0,97.105.19.58,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-07,2018-09-07,08:58:58,spring,250,24.0,97.105.19.58,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-07,2018-09-07,08:59:05,spring/fundamentals/services,250,24.0,97.105.19.58,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-07,2018-09-07,14:14:46,spring/fundamentals/controllers,250,24.0,97.105.19.58,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-07,2018-09-07,14:15:00,spring/fundamentals/views,250,24.0,97.105.19.58,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-07,2018-09-07,14:57:50,spring/fundamentals/services,250,24.0,97.105.19.58,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-07,2018-09-07,14:57:56,spring/fundamentals/form-model-binding,250,24.0,97.105.19.58,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-10,2018-09-10,09:04:21,html-css/elements,250,24.0,97.105.19.58,Voyageurs,2018-05-29,2018-10-11,Java
2018-09-10,2018-09-10,09:04:39,spring/fundamentals/repositories,250,24.0,97.105.19.58,Voyageurs,2018-05-29,2018-10-11,Java


In [11]:
#24 seems to have completed the course but barely looked at the curriculum - Java

#858 started using a month plus in and then stopped after a week

#41, 371, 741, 56, 57, 45, 36, 250 only started accessing the curriculum when the cohort was almost finished - Java

In [12]:
#The below users accessed the curriculum on the first day (or first week) and then stopped. I assume these are students who dropped out.
#Java
#879, 832,  388, 572, 772, 529, 543, 46,
#981 - listed as Neptune, but seems like it could have been Oberon

#Oberon Cohort(968, 963, 956, 961, 972, 973, 967, 959, 971, 964, 960, 978, 965, 962, 969, 975)
#No usage past 2021-04-21, but cohort end_date == 2021-10-01

#Data Science
#679, 

#487 only accessed the appendix pages one day during the first month of classes and that's it. Also, data science

In [43]:
active_student_access[active_student_access.name == 'Oberon'].user_id.value_counts()

966    159
970    113
957    112
979    104
958     95
954     88
955     80
977     76
974     76
975     66
969     62
962     52
965     51
978     49
960     45
964     42
971     41
959     38
967     33
973     32
972     31
976     25
961     18
963     17
968     16
956      5
Name: user_id, dtype: int64

## 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?